In [1]:
import os, pyodbc
import pandas as pd
#import numpy as np
from pandas import DataFrame
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from sqlalchemy import create_engine


In [2]:
%pylab


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\python27\arcgis10.2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Connect to SQL db and create dataframe

In [3]:
engine = create_engine("mssql+pyodbc://gt_KL_Logger")


Query for Au standards

In [4]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
COALESCE(c.Col4, c.col65,c.col45) as Au_gpt, d.DateForagFin as 'Date', c.Description as 'Desc' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblColletInformation d on b.idkey = d.idNuSondage \
where  c.TypeQAQC = 3 and COALESCE(c.Col4, c.col65, c.Col45) is not NULL \
and isnumeric(COALESCE(c.Col4, c.col65, c.Col45)) = 1 and d.DateForagFin \
is not NULL and a.NomProjet like 'UpperCanada%'"

In [5]:
engine2 = create_engine("mssql+pyodbc://gt_KL_Master")

In [6]:
sql2 = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', e.DateEmis as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
join tblCertEchQAQC d on c.idkey = d.idQAQC \
join tblAnalRegCertificat e on d.NuCertificat = e.NuCertificat \
where c.TypeQAQC = 3 and c.Col101 is not NULL \
and a.NomProjet = 'UpperCanada'  and isnumeric(Col101) = 1"

Query for Blanks

In [54]:
sql = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
COALESCE(c.Col4, c.col65, c.Col45) as Au_gpt, d.DateForagFin as 'Date', c.Description as 'Desc'\
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
left join tblColletInformation d on b.idkey = d.idNuSondage \
where c.TypeQAQC = 1 and a.NomProjet like 'UpperCanada%' and COALESCE(c.Col4, c.col65, c.col45) is not NULL \
and COALESCE(c.Col4, c.col65, c.Col45) <> '' \
and d.DateForagFin is not NULL "

In [55]:
sql2 = "Select  b.Nom as 'HOLEID', c.NuEchantillon as 'SAMPLEID', c.Reference as 'STD', \
c.Col101 as 'Au_gpt', d.DateForagFin as 'Date' \
from tblNomProjet a join tblNuSondage b on a.idkey = b.idNomProjet \
join tblQAQC c on b.idkey = c.idNuSondage \
left join tblColletInformation d on b.idkey = d.idNuSondage \
where c.TypeQAQC = 1 and c.Col101 is not NULL \
and a.NomProjet = 'UpperCanada'  and isnumeric(Col101) = 1"

In [7]:
df = pd.read_sql(sql, engine)#, index_col = 'Date')
df2 = pd.read_sql(sql2, engine2)#, index_col = 'Date')

#df = df.append(df2)

In [8]:
stds = pd.read_sql("Select * from tblTypeStandard", engine)
stds.columns = ['idkey','std_name','alias','col','avg','active','sigma']

In [9]:

df['Au_val'] = df.Au_gpt[(df.Au_gpt <> '<0.005') & (df.Au_gpt <> '< 0.01') & (df.Au_gpt <> '<0.01')].astype(np.float)
df = df.fillna({'Au_val':0.005})
df2['Au_val'] = df2.Au_gpt[(df2.Au_gpt <> '<0.005') & (df2.Au_gpt <> '< 0.01') & (df2.Au_gpt <> '<0.01')].astype(np.float)
df2 = df2.fillna({'Au_val':0.005})

In [10]:
df = df.append(df2)

In [11]:
df.index = df.Date

In [12]:
#df.STD = "BLANK"
groups =list(set(df['STD']))

Generate Standard Graph

In [17]:
# with plt.xkcd():
fig = plt.figure(facecolor = 'w', figsize = (10, 12), tight_layout = False)

# fig.suptitle('CRM Analyses for week of %s' % (datetime.date.today() + datetime.timedelta(4 - \
#               datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 16)
#fig.figimage(logo, 700,1025, zorder=1)
length = len(groups[:2])

for a, g in enumerate(groups[0:2]):   
    ax1 = fig.add_subplot(length,1,a+1)
    time_buffer = max((df.index.max() - df.index.min()) * 0.1, timedelta(5))
    datarange = [df.index.min() - time_buffer, df.index.max()+ time_buffer]

    #standard graphs
    std_mean = stds.avg[stds['alias'] == g].values[0]
    std_sd = stds.sigma[stds['alias'] == g].values[0]
    ax1.plot(datarange, [df[df.STD == g].Au_val.mean(), df[df.STD == g].Au_val.mean()], 'r-',\
              label = 'measured avg = %0.3f' % (df[df.STD == g].Au_val.mean()), linewidth = 1)
    ax1.plot(datarange, [std_mean, std_mean], 'b-', label = 'published avg = %0.3f' % (std_mean), linewidth = 1)
    _2sig1 = df[df.STD == g].Au_val.mean() + 2 * df[df.STD == g].Au_val.std()
    _2sig2 = df[df.STD == g].Au_val.mean() - 2 * df[df.STD == g].Au_val.std()
    _3sig1 = df[df.STD == g].Au_val.mean() + 3 * df[df.STD == g].Au_val.std()
    _3sig2 = df[df.STD == g].Au_val.mean() - 3 * df[df.STD == g].Au_val.std()
    _2sigm1 = std_mean + 2 * std_sd
    _2sigm2 = std_mean - 2 * std_sd
    _3sigm1 = std_mean + 3 * std_sd
    _3sigm2 = std_mean - 3 * std_sd
    ax1.plot(datarange, [_2sig1,_2sig1], 'r--', label = u'measured 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sig2,_2sig2], 'r--')
    ax1.plot(datarange, [_3sig1,_3sig1], 'r:', label = u'meausred 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sig2,_3sig2], 'r:')
    ax1.plot(datarange, [_2sigm1,_2sigm1], 'b--', label = u'published 2 sigma', linewidth = 1)
    ax1.plot(datarange, [_2sigm2,_2sigm2], 'b--')
    ax1.plot(datarange, [_3sigm1,_3sigm1], 'b:', label = u'published 3 sigma', linewidth = 1)
    ax1.plot(datarange, [_3sigm2,_3sigm2], 'b:', linewidth = 2)
    ax1.plot(df.Au_val[df.STD == g], 'wo', label = '_nolegend_')
    ax1.plot(df.Au_val[df.STD == g], 'k.', label = '_nolegend_')
#     labels = df['SAMPLEID'][(df.STD == g) & ((df.Au_val < std_mean - 2 * std_sd) |\
#                          (df.Au_val > std_mean + 2 * std_sd))]
#     for d, l in labels.iteritems():
#         ax1.annotate(l, xy = (d, df.Au_val[df.SAMPLEID == l]), xytext=(5, 5), textcoords= 'offset points', color = 'r')
    ylim = ax1.get_ylim()
    yshift = (ylim[1] - ylim[0]) /10.0

    ax1.set_ylim(ylim[0] - yshift, ylim[1] + yshift)
    ax1.legend(loc = 'best', fontsize = 'small')
    ax1.set_title('Analyses for %s, n= %d' % (g, df[df.STD == g].Au_val.count()), fontsize = 12)

    ax1.set_ylabel('Au g/t')
    xticks = ax1.get_xticks()
    date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
    ax1.set_xticklabels([])
    if a == length -1:
      ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
    
#fig.savefig('C:\\Temp\\crm.jpg', dpi = 200)  




Generate Blank graph

In [77]:
# with plt.xkcd():
fig = plt.figure(facecolor = 'w', figsize = (10, 8))


# fig.suptitle('Blank Analyses for week of %s' % (datetime.date.today() + datetime.timedelta(4 - \
#               datetime.date.weekday(datetime.date.today()))).strftime('Friday, %B %d, %Y'), fontsize = 16)
#fig.figimage(logo, 690*3,1035*3, zorder=1)
length = len(groups)

for a, g in enumerate(groups): 
    ax1 = fig.add_subplot(length,1,a+1)

    time_buffer = max((df.index.max() - df.index.min()) * 0.1, timedelta(5))
    datarange = [df.index.min() - time_buffer, df.index.max()+ time_buffer]

    ax1.plot(datarange, [0.015, 0.015], 'r-', label = '3 * L.O.D. (0.015 g/t)', linewidth = 1.5)
    _80pct = df[df.STD == g].Au_val.quantile(0.8)
    _90pct = df[df.STD == g].Au_val.quantile(0.9)
    ax1.plot(datarange, [_80pct,_80pct], 'b--', label = u'80th percentile', linewidth = 1.5)
    ax1.plot(datarange, [_90pct,_90pct], 'b:', label = u'90th percentile', linewidth = 1.5)
    ax1.plot(df.Au_val[df.STD == g], 'wo', label = '_nolegend_')
    ax1.plot(df.Au_val[df.STD == g], 'k.', label = '_nolegend_')
    labels = df[df.STD == g].SAMPLEID[(df[df.STD == g].Au_val > 0.015)]
#     for d, l in labels.iteritems():
#         ax1.annotate(l, xy = (d, df.Au_val[df.SAMPLEID == l]), xytext=(5, 5), textcoords= 'offset points')
    ylim = ax1.get_ylim()
    yshift = (ylim[1] - ylim[0]) /10.0

    ax1.set_ylim(ylim[0] - yshift, ylim[1] + yshift)
    ax1.legend(loc = 'best', fontsize = 'small')
    ax1.set_title('Analyses for %s, n= %d' % (g, df[df.STD == g].Au_val.count()), fontsize = 12)

    ax1.set_ylabel('Au g/t')

    ax1.set_xticklabels([])
    if a == length -1:    
        xticks = ax1.get_xticks()
        date_labels = [datetime.date.fromordinal(int(f)) for f in xticks]
        ax1.set_xticklabels([x.strftime('%d-%b-%y') for x in date_labels], rotation = 45)
    
#fig.savefig('C:\\Temp\\blank.jpg', dpi = 200)  


In [78]:
df[df.Au_val > 0.49]

,HOLEID,SAMPLEID,STD,Au_gpt,Date,Au_val
Date,,,,,,
2010-04-30,UC10_076,_UC10_076_60375,BLANK,1.13,2010-04-30,1.130
2011-09-01,UC11_367,_UC11_367_206075,BLANK,3.27,2011-09-01,3.270
2012-03-13,UC12_428,_UC12_428_M571525,BLANK,1.96,2012-03-13,1.960
2012-08-18,UC12_457,_UC12_457_N592250,BLANK,0.696,2012-08-18,0.696
2013-04-30,UC13_018E,_UC13_018E_Q555900,BLANK,0.52,2013-04-30,0.520
2013-04-23,UC13_173E,_UC13_173E_P965250,BLANK,0.504,2013-04-23,0.504
2013-04-11,UC13_174E,_UC13_174E_P964525,BLANK,1.04,2013-04-11,1.040
2013-03-29,UC13_474E,_UC13_474E_P964175,BLANK,2.11,2013-03-29,2.110
2013-03-26,UC13_478,_UC13_478_L765775,BLANK,0.504,2013-03-26,0.504


In [62]:
grp = df.groupby('STD')


In [52]:
fig = plt.figure(facecolor = 'w')
ax = fig.add_subplot(1,1,1)
colors = ['#003300','#006633','#FF0000','#00FF00','#0000FF','#FFFF00','#FF00FF','#00FFFF','#666666','#FF9966', '#006666','#336699','#996633']

In [63]:
for a, g in enumerate(grp):
    #if g[0] in ['STD11','STD9']:
    ax.plot(g[1].Au_val, 'wo', label = g[0])#, c = colors[a])

In [64]:
ax.legend(loc='upper left', fontsize = 'small')

In [97]:
len(grp)

10

In [20]:
ax.plot(df[df.Desc.str.startswith('Corrected', na=False)].Au_val, 'b.', label = '_no_legend_')

In [19]:
ax.plot(df[~df.Desc.str.startswith('Corrected', na=False)].Au_val, 'r.', label = '_no_legend_')

In [19]:
ax.legend(loc = 'upper left', fontsize = 'small')

In [65]:
for label in  ax.xaxis.get_majorticklabels():
    label.set_rotation(40)

In [66]:
ax.set_ylabel('Au g/t')

In [67]:
ax.set_title('UC Control Sample Distribution from 2009 to 2015')

In [48]:
len(grp)

13

In [18]:
groups

[u'CM28',
 u'CM24',
 u'SL77',
 u'STD19',
 u'STD9',
 u'STD18',
 u'STD15',
 u'STD4',
 u'STD7',
 u'STD11',
 u'STD17',
 u'STD13',
 u'STD12']